# 3D Scan Visualisation

This experiments with plotting 3D scan data with various python tools including:

- Open3d (requires python < 3.9.5 so less useful - and does not integrate with Qt nicely)
- Matplotlib
- pyqtgraph

Overall pyqtgraph looks to be the most appropriate method for plotting and will integrate well with a GUI interface.

In [ ]:
from pathlib import Path

%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from ai_ct_scans.data_loading import MultiPatientLoader
import pyqtgraph

## Load Data

In [ ]:
data = MultiPatientLoader()

In [ ]:
data.patients[0].abdo.scan_1.load_scan()
print(data.patients[0].abdo.scan_1.full_scan.shape)

## Normalise Data

In [ ]:
norm_data = data.patients[0].abdo.scan_1.full_scan
norm_data *= 1.0 / norm_data.max()

In [ ]:
print(norm_data[100:110, 255, 100:101])

## Try pyqtgraph plotting

In [ ]:
# Threshold and slice data
pointcloud = np.where(norm_data[:, :, :] > 0, norm_data[:, :, :], 0)
print(pointcloud.shape)

In [ ]:
%gui widget
from PySide2.QtWidgets import QApplication
from PySide2 import QtCore
import pyqtgraph as pg
import pyqtgraph.opengl as gl

In [ ]:
# start qt event loop
_instance = QApplication.instance()
if not _instance:
    _instance = QApplication([])
app = _instance

QApplication.setAttribute(QtCore.Qt.AA_EnableHighDpiScaling)
QApplication.setAttribute(QtCore.Qt.AA_UseHighDpiPixmaps)

pg.setConfigOptions(antialias=True)

In [ ]:
# Data (for only plotting whole scan)
points = pointcloud

d2 = np.empty(points.shape + (4,), dtype=np.ubyte)

intensity_vals = points * (255.0 / (points.max() / 1))

d2[..., 0] = intensity_vals
d2[..., 1] = intensity_vals * 0
d2[..., 2] = intensity_vals * 0

d2[..., 3] = intensity_vals
d2[..., 3] = (d2[..., 3].astype(float) / 255.0) ** 2 * 255 * 0.1

# Subset of data
d_vals = np.where(norm_data[:, :, :] > 0.3, norm_data[:, :, :], 0)

d_extra = np.empty(d_vals.shape + (4,), dtype=np.ubyte)

intensity_vals = d_vals * (255.0 / (d_vals.max() / 1))

d_extra[..., 0] = intensity_vals
d_extra[..., 1] = intensity_vals * 0
d_extra[..., 2] = intensity_vals * 0

d_extra[..., 3] = intensity_vals
d_extra[..., 3] = (d_extra[..., 3].astype(float) / 255.0) ** 2 * 255 * 0.1

# Subset of data2
d_vals = np.where(
    np.logical_and(norm_data[:, :, :] < 0.3, norm_data[:, :, :] > 0.1),
    norm_data[:, :, :],
    0,
)

d_other = np.empty(d_vals.shape + (4,), dtype=np.ubyte)

intensity_vals = d_vals * (255.0 / (d_vals.max() / 1))

d_other[..., 0] = intensity_vals * 0
d_other[..., 1] = intensity_vals
d_other[..., 2] = intensity_vals * 0

d_other[..., 3] = intensity_vals
d_other[..., 3] = (d_other[..., 3].astype(float) / 255.0) ** 2 * 255 * 0.1

In [ ]:
# Combine extra data
d2 = d_other + d_extra

# Add Axis lines
d2[:, 0, 0] = [255, 0, 0, 255]
d2[0, :, 0] = [0, 255, 0, 255]
d2[0, 0, :] = [0, 0, 255, 255]

In [ ]:
# Create view widget
view = gl.GLViewWidget()
# view.orbit(256, 256)
# view.setCameraPosition(pos=[0,0,0], distance=100, azimuth=180, elevation=40)
view.show()
view.setWindowTitle("pyqtgraph: GLVolumeItem CT Scans")

# Optional Grid
# g = gl.GLGridItem()
# g.scale(20, 20, 1)
# view.addItem(g)

# Set scan data
scatter = pg.opengl.GLVolumeItem(d2, smooth=False, glOptions="translucent")
scatter.translate(-d2.shape[0] / 2, -d2.shape[1] / 2, -150)
view.addItem(scatter)

# Trigger App
sys.exit(app.exec_())

### pyqtgraph scatter plot

Code below attempts to form a suitable data shape for scatter plotting and to centre the scatter plot.

In [ ]:
pointcloud = np.where(norm_data[:, 250:255, :] > 0, norm_data[:, 250:255, :], 0)

# Convert to nonzero point indices for plotting
x, y, z = pointcloud.nonzero()

# Assign colour intensity values
c = pointcloud[x, y, z]
colour_points = np.column_stack((c, c, c, 0.05 * np.ones(len(c))))

# Center points around centre of scan
x = x - 0.5 * max(x)
y = y - 0.5 * max(y)
z = z - 0.5 * max(z)

points = np.column_stack((x, y, z))
colours = np.column_stack((np.ones(len(c)), np.ones(len(c)), np.ones(len(c)), c))

print(points.shape)

In [ ]:
# Check points formatting
print(points.shape)
print(points)

# Create view widget
view = gl.GLViewWidget()
scatter = pg.opengl.GLScatterPlotItem()

# Set scan data
scatter.setData(pos=points, size=1, color=colour_points, pxMode=True)
view.addItem(scatter)
view.show()

sys.exit(app.exec_())

## Alternative plotting methods

### Generate slice of data (for alternative plotting)

In [ ]:
view_of_data = np.where(norm_data[:, 254:255, :] > 0.5, True, False)

### Plot data in 3D with matplotlib

In [ ]:
slice_of_data = norm_data[:, 254:255, :] - norm_data.min()

x, y, z = slice_of_data.nonzero()
cval = slice_of_data[x, y, z].flat

fig = plt.figure(figsize=(4, 3), dpi=300)
ax = fig.add_subplot(projection="3d")
s = ax.scatter(x, y, z, c=cval, alpha=0.9, s=0.001)
plt.ylim([-5, 5])

ax.view_init(0, 90)
plt.savefig(Path.cwd() / "3d_scan_plot_scatter_test.png")
plt.show()

Try different slice

In [ ]:
slice_of_data = norm_data[250:400, 225:255, 250:400] - norm_data.min()

x, y, z = slice_of_data.nonzero()
cval = slice_of_data[x, y, z].flat

fig = plt.figure(figsize=(4, 3), dpi=300)
ax = fig.add_subplot(projection="3d")
s = ax.scatter(x, y, z, c=cval, alpha=0.2, s=0.001)

ax.view_init(30, 60)
plt.savefig(Path(os.getcwd()) / "3d_scan_plot_scatter_test2.png")
plt.show()

Try voxel plotting

In [ ]:
view_of_data = norm_data[:, 254:255, :] - norm_data.min()

fig = plt.figure()
ax = fig.add_subplot(projection="3d")
ax.voxels(view_of_data)
plt.ylim([-10, 10])

ax.view_init(30, 60)
plt.savefig(Path.cwd() / "3d_scan_plot_test.png")
plt.show()

### Try Open3D Rendering

In [ ]:
import open3d as o3d  # Does not install with python 3.9.5 (needs older version)

In [ ]:
pcd = o3d.geometry.PointCloud()

In [ ]:
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colours)
o3d.io.write_point_cloud(Path.cwd() / "sync.ply", pcd)

In [ ]:
pcd_load = o3d.io.read_point_cloud(Path.cwd() / "sync.ply")

In [ ]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd_load)
vis.get_render_option().load_from_json(Path.cwd() / "renderoption.json")
vis.run()
vis.destroy_window()

In [ ]:
renderer = o3d.visualization.rendering.OffscreenRenderer(1024, 768, headless=True)